In [2]:
''' main hashing functions
'''
import string
from collections import defaultdict

import numpy as np

In [3]:
def symbol_encode(n, symbols):
    nb_symbols = len(symbols)
    
    if n < nb_symbols:
        return symbols[n]
    
    retval = ''
    while n > 0:
        n, k = divmod(n, nb_symbols)
        retval += symbols[k]

    return retval


In [4]:
def normalize(v, ord=2):
    norm = np.linalg.norm(v, ord=ord)
    if not norm:
        return v
    inv_norm = 1./norm
    return inv_norm*v


In [5]:
class HashGenerator(object):
    def __init__(self, symbols=None):
        self.curr = 0
        self.symbols = symbols

        if self.symbols is None:
            self.symbols = string.digits+string.ascii_lowercase
            

    def __call__(self):
        retval = symbol_encode(self.curr, self.symbols)
        self.curr += 1
        return retval


In [6]:
import math

class ThresholdingHasher(object):
    def __init__(self, r, m, ord=None):
        self.r = r
        self.m = m
        self.hashes = defaultdict(HashGenerator())
        self.ord = ord

    @property
    def h(self):
        return math.sqrt(2*self.r*np.log(self.m))
    
    def encode(self, vector):
        if self.ord:
            vector = normalize(vector, self.ord)
        u = self.transform(vector)
        keys = np.flatnonzero(u > self.h)
        return [self.hashes[k] for k in keys]


In [7]:
class GaussianHasher(ThresholdingHasher):
    def __init__(self, d, m, r, ord=None, random_state=np.random.RandomState(42)):
        super(GaussianHasher, self).__init__(r, m, ord)
        self.matrix = random_state.normal(size=(m, d))

    def transform(self, vector):
        return self.matrix.dot(vector)

In [18]:
class FourierHasher(ThresholdingHasher):
    ''' Approximate the random Gaussian mapping by a sparse operation followed
        by a Fourier transform.
        The vector is duplicated m/d times, and each element of the duplicated
        vector is multiplied randomly by +1 or -1.  These duplicated vectors are
        concatenated and then their DFT is returned.  This entire operation
        approximates multiplication of the vector by a standard normal Gaussian 
        m x d matrix.
        Note that a dct implementation must be provided to the constructor.  Two
        options are:
            * scipy.fft.dct
            * pyfftw.interfaces.scipy_fftpack.dct
        See: https://arxiv.org/abs/1507.05929
    '''
    def __init__(self, d, m, r, ord=None, dct=None, random_state=np.random.RandomState(42)):
        from scipy.fftpack import dct
        if m%d:
            raise ValueError("m must be an integer multiple of d")

        @property
        def h(self):
            return math.sqrt(2*r*np.log(m))
        
        super(FourierHasher, self).__init__(r, m, ord)
        self.signs = random_state.choice([-1.0, 1.0], size=(int(m/d), d))
        self.sqrt_d = np.sqrt(d)

        self.dct = dct
        if not self.dct:
            raise NotImplementedError("a dct implmentation must be provided")

    def transform(self, vector):
        return self.dct(np.multiply(self.signs, vector).flatten(), type=2, norm='ortho')*self.sqrt_d

In [21]:
'''
m = 10000
r = 0.4
d = 4
h = math.sqrt(2*r*np.log(m))

x1 = FourierHasher(d,m,r)
v1 = [-2, 3, -2, 4] 
v2 = [-10, 5, -4, 5] 
tr1 = x1.encode(v1)
tr2 = x1.encode(v2)

print(tr1)
#print("\n \n")
print(len(tr2))

print(x1.hashes)
print("\n \n")
print(x1.hashes)
#[print("\n \n")]
#print(x2.hashes.keys())

#ha = HashGenerator()
#print(ha())

'''

'\nm = 10000\nr = 0.4\nd = 4\nh = math.sqrt(2*r*np.log(m))\n\nx1 = FourierHasher(d,m,r)\nv1 = [-2, 3, -2, 4] \nv2 = [-10, 5, -4, 5] \ntr1 = x1.encode(v1)\ntr2 = x1.encode(v2)\n\nprint(tr1)\n#print("\n \n")\nprint(len(tr2))\n\nprint(x1.hashes)\nprint("\n \n")\nprint(x1.hashes)\n#[print("\n \n")]\n#print(x2.hashes.keys())\n\n#ha = HashGenerator()\n#print(ha())\n\n'